In [18]:
import os
import pydicom
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.models import Model
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt

In [ ]:
DATASET_PATH = "C:/Users/jrsl2/Downloads/archive/CT_low_dose_reconstruction_dataset"

def load_dicom_images(folder, size=(128, 128)):
    images = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith((".IMA", ".dcm")):  
                path = os.path.join(root, file)
                try:
                    dicom_data = pydicom.dcmread(path)
                    image = dicom_data.pixel_array
                    image = cv2.resize(image, size)
                    image = image.astype(np.float32) / np.max(image)  
                    images.append(image)
                except Exception as e:
                    print(f"Error {path}: {e}")
    print(f" {folder}: {len(images)}")
    return np.array(images)

original_data_path = os.path.join(DATASET_PATH, "Original Data")

quarter_dose_images = None
full_dose_images = None

for dose_type in ["Quarter Dose", "Full Dose"]:
    dose_path = os.path.join(original_data_path, dose_type)
    images = load_dicom_images(dose_path) 

    if dose_type == "Quarter Dose":
        quarter_dose_images = images
    else:
        full_dose_images = images 

print(f"{quarter_dose_images.shape}")
print(f"{full_dose_images.shape}")


quarter_dose_images = np.expand_dims(quarter_dose_images, axis=-1)
full_dose_images = np.expand_dims(full_dose_images, axis=-1)

print(f"{quarter_dose_images.shape}")
print(f"{full_dose_images.shape}")



Imagens Quarter Dose: (16628, 128, 128)
Imagens Full Dose: (16628, 128, 128)
Quarter Dose após expandir: (16628, 128, 128, 1)
Full Dose após expandir: (16628, 128, 128, 1)
